# Automation for IB portfolios

Latest version: 2024-08-27  
Author: MvS

## Description

Notebook to automate calculation and setting of stop-loss in IB portfolios

## Result


### Warning: This notebook will place live orders


In [ ]:
from ib_async import *
import yfinance as yf

import pandas as pd
import numpy as np
from dotenv import dotenv_values

import logging
import datetime
import time

# log everything
if True:
    util.logToConsole(logging.DEBUG)

# ib_sysnc: notebook relevant
util.startLoop()

# read env vars
env_dict = dotenv_values("../.env")

ib = IB()
ib.connect(env_dict['IB_API_IP'], int(env_dict['IB_API_PORT']), clientId=int(env_dict['IB_API_KEY']))


### Get liquidation value of whole account

In [ ]:
logger = logging.getLogger(__name__)

liq_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

logger.info('Printing liquidation value:')
for counter, (acc, bal, curr) in enumerate(liq_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

### Unpack cash and stock balance

In [ ]:
cash_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'CashBalance' and v.currency != 'BASE']
stock_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'StockMarketValue' and v.currency != 'BASE']

logger.info('Printing cash balance:')
for counter, (acc, bal, curr) in enumerate(cash_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

logger.info('Printing stock balance:')
for counter, (acc, bal, curr) in enumerate(stock_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")
    
#ib.accountValues()

### Unpack position balance

In [ ]:
logger.info('Printing open positions:')
portfolio_pos = []

for counter, position in enumerate(ib.positions(), 1):

    contract = position.contract
    position_type = 'LONG' if position.position > 0.0 else 'SHORT'

    logger.info(f"{position.account}, {counter:0d}: {contract.conId}, {contract.secType},\
        {position_type}, {contract.symbol}, {contract.localSymbol},\
        {contract.exchange}, {contract.currency}, {position.position:8.2f},\
        {position.avgCost:8.2f}")

    portfolio_pos.append(
        [
            position.account,
            contract.conId,
            contract.secType,
            position_type,
            contract.symbol,
            contract.localSymbol,
            contract.exchange,
            contract.currency,
            position.position,
            position.avgCost,
        ]
    )

columns=['account', 'conId', 'secType', 'position_type', 'symbol', 'localSymbol', 'exchange', 'currency', 'position', 'avgCost']
portfolio_df = pd.DataFrame(portfolio_pos, columns=columns)

del portfolio_pos, contract, position_type, columns
portfolio_df

### Constructing the symbol for yahoo finance

The list of mappings needs to be updated as soon as new securities from different exchanges are traded

In [ ]:

exchange_map = {
    'IBIS': 'DE',
    'GETTEX2': 'MU'
}

def yf_symbol(x):
    ex = x['exchange']
    ex_sym = x['exchange_sym']
    sym = x['symbol']
    loc_sym = x['localSymbol']

    if not pd.isna(ex_sym):
        return f"{sym}.{ex_sym}"
    else:
        return f"{sym}"

portfolio_df['exchange_sym'] = portfolio_df['exchange'].map(exchange_map, na_action='ignore')
portfolio_df['yf_sym'] = portfolio_df.apply(yf_symbol, axis=1)
portfolio_df

In [ ]:
periods = 10
dt_end = datetime.datetime.today()
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
dt_data_start = dt_end - datetime.timedelta(days=periods*2)

stock_data_list = []
for symbol in portfolio_df['yf_sym'].to_list(): 
    time.sleep(1)
    try:
        # Grab sufficient stock data for averaging SMAs
        load_df = yf.download(
            symbol,
            start=dt_data_start.strftime('%Y-%m-%d'),
            end=dt_end.strftime('%Y-%m-%d'),
            progress=False,
        )

        assert load_df.shape[1] == 6 and load_df.shape[0] >= periods
        stock_data_list.append((symbol, load_df))

    except AssertionError:
        print(f"Download failed for symbol {symbol}.  Skipping...")

In [ ]:
for (bla, cs) in stock_data_list:
    print(bla)

In [ ]:
cs.shape

In [ ]:
cs['Adj Close'].iloc[-1]

In [ ]:
from importlib import reload  # Python 3.4+
import utils.atr_utils as atr

reload(atr)

bla = atr.calc_atr_spikes(cs, periods=14)


In [ ]:
ib.disconnect()

Create a contract and a market order:

In [ ]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.11, conditionsIgnoreRth=True)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [ ]:
trade = ib.placeOrder(contract, order)

### List orders and cancel an order

In [ ]:
orders = ib.openOrders()
orders

In [ ]:
orders

In [ ]:
ib.cancelOrder(order=orders[1] )

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [ ]:
ib.sleep(1)
trade.log

``trade`` will also available from ``ib.trades()``:

In [ ]:
assert trade in ib.trades()

Likewise for ``order``:

In [ ]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [ ]:
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

``status`` will change from "PendingSubmit" to "Submitted":

In [ ]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [ ]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [ ]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

And now cancel it:

In [ ]:
ib.cancelOrder(limitOrder)

In [ ]:
limitTrade.log

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [ ]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

What are our positions?

In [ ]:
ib.positions()

What's the total of commissions paid today?

In [ ]:
sum(fill.commissionReport.commission for fill in ib.fills())

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [ ]:
order = MarketOrder('SELL', 20000)
ib.whatIfOrder(contract, order)

In [ ]:
ib.disconnect()